<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0410iwa_yoshi_presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 岩下，吉原勉強会資料

- 文責: 浅川伸一 <askaawa@ieee.org>
- date: 2022_0410
- filename: `2022_0410iwayoshi_ja_edu.ipynb`


In [4]:
from google.colab import files
uploaded = files.upload()  # `20220324_minnichi_goilist_2202.xlsx` を指定してアップロードする

Saving 20220324_minnichi_goilist_2202.xlsx to 20220324_minnichi_goilist_2202.xlsx


In [ ]:
%%time
import sys

# 次行はローカルな実行環境とクラウド計算環境である google colab との差分を吸収するため
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install jaconv
    !pip install japanize_matplotlib
    !git clone https://github.com/ShinAsakawa/ccap.git > /dev/null 2>&1
    !pip install 'konoha[mecab]'
    
import matplotlib.pyplot as plt
import japanize_matplotlib

In [1]:
import pandas as pd
import jaconv
import unicodedata

excel_filename = '20220324_minnichi_goilist_2202.xlsx'
a = pd.read_excel(excel_filename)

# 岩下先生からいただいたエクセルファイルの ['ことば'] 列の単語には，
# 末尾に空白文字が入っているようなので  `" ".join(word.split())` して除去
min2022_0327 = [" ".join(unicodedata.normalize('NFKC',word).split()) for word in sorted(list(a['ことば']))]


In [ ]:
# 上で読み込んだデータを minnichi 辞書として登録
minnichi = {}
minnichi_vocab = []
for l, _w in zip(a.iterrows(), min2022_0327):
    num = int(l[0])
    _word = _w
    _class = l[1][1]
    _pos = l[1][2]
    minnichi[_word] = {'num':num, 
                       #'word': _word,
                       '課':_class, 
                       '品詞':_pos}
    minnichi_vocab.append(_word)

print(f'読み込んだデータ数:{len(minnichi)}')
minnichi_vocab =  sorted(set(minnichi_vocab))
print(f'でも単語数としては:{len(minnichi_vocab)} です。重複があるのかな？' )
a

In [ ]:
# 前回お話した，ユニコードの正規化についてです。
# 実際の動作とは関係ありません。
# NFC, NFKC, NFD, NFKD と 4 種類があります。
# それぞれの意味は，以下のとおりです
# NF: Normalized Form
# C: コンポーズド  飾り記号を分けて考えない
# D: デコンポーズド 飾り記号を分けて考える
# K: 互換性を保証する Comaptibility の意味。だが C が composed と競合するので K にしている
import unicodedata
help(unicodedata.normalize)

In [ ]:
print(minnichi_vocab)

In [ ]:
%%time
# Google Colaboratory 上で実行する場合に，必要となるライブラリをインストールする
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install transformers > /dev/null 2>&1 

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1

In [12]:
# 新しい minnichi を MeCab にしてみる。
import MeCab
mcb = MeCab.Tagger().parse

# MeCab の品詞分類のデフォルト設定は [IPADIC 2.7](https://chasen.naist.jp/snapshot/ipadic/ipadic/doc/ipadic-ja.pdf) 
# に基づいている。
# [IPADIC 大分類](https://hayashibe.jp/tr/mecab/dictionary/ipadic) は以下の通り
pos_ipa = ['名詞', '接頭詞', '動詞', '形容詞', '副詞', '連体詞', '接続詞', '連体詞', 
           '接続詞', '助詞', '助動詞', '感動詞', '記号', 'フィラー', 'その他']

for wrd in minnichi.keys():
    wrd_splited = mcb(wrd).strip().splitlines()[:-1]
    _mcb_data = []
    for _wrd in wrd_splited:
        x = _wrd.split('\t')
        _x = x[1].split(',')
        __x = {'表層形':x[0], 
              '原形':_x[7],
              #'品詞':_x[4],
              '品詞':_x[4].split('-')[0],
              '品詞1':_x[5],
              '品詞2':_x[6]
              }
        _mcb_data.append(__x)
    minnichi[wrd]['mecab'] = _mcb_data

#minnichi

In [13]:
from termcolor import colored
import numpy as np

for i in range(10):
    wrd = minnichi_vocab[np.random.choice(len(minnichi))]
    print(colored(wrd,'blue', attrs=['bold']), end=": ") 
    #print(colored(minnichi[i]['word'],'blue', attrs=['bold']), end=": ") 
    for _i in minnichi[wrd]['mecab']:
        print('表層形',colored(_i['表層形'], 'green', attrs=['bold']),
              '品詞',  colored(_i['品詞'],  'green', attrs=['bold','blink']), end=" ") 
    print()

習います: 表層形 習い 品詞 五段 表層形 ます 品詞 助動詞 
終わります: 表層形 終わり 品詞 五段 表層形 ます 品詞 助動詞 
銀行: 表層形 銀行 品詞  
邪魔: 表層形 邪魔 品詞  
にぎやか: 表層形 にぎやか 品詞  
失礼します: 表層形 失礼 品詞  表層形 し 品詞 サ行変格 表層形 ます 品詞 助動詞 
菓子: 表層形 菓子 品詞  
こんにちは: 表層形 こんにちは 品詞  
台所: 表層形 台所 品詞  
地震: 表層形 地震 品詞  


## 若干の老婆心


In [ ]:
# ここで MeCab の使い方を簡単に紹介する。
# 入力文を分かち書きしたいだけならば，以下のように '-Owakati' オプションを指定して
# MeCab を呼び出せばよい。
print(MeCab.Tagger('-Owakati').parse('吾輩は猫である').strip())

In [ ]:
# あるいは入力文を変数に代入して実行する
s = '名前はまだない'
print(MeCab.Tagger('-Owakati').parse(s).strip())

# 最後に付いている `strip()` は，文字列の最終要素，この場合は改行コード，を取り去るためである。

In [ ]:
# 何度も分かち書きを繰り返して呼び出すのであれば，あらかじめ分かち書きを定義しておく
wakati = MeCab.Tagger('-Owakati').parse
s2 = 'どこで生れたか頓と見当がつかぬ。'
print(wakati(s2).strip())

In [ ]:
# 上記のようにして分かち書きした文は，空白で区切られているので，分割するには `split(' ')` を用いる
# `split(' ')` で分割された文は，リストになっているので，表示の際はカギカッコで囲まれている。
print(wakati(s2).strip().split(' '))

In [ ]:
# ここで，みんなの日本語データが一行に一文で文字列からなるリストであると仮定しよう。
# 以下のようにである:
minnichi_sentences = [
'ジュースをお願いします。',
'いらっしゃいませ。メニューです。どうぞ。',
'いくらですか。',
'1.ホン:カレーとコーヒーをください。',
'2.ジル:サンドイッチとジュースをお願いします。']

# 上記のデータ `minnichi_sentences` を分かち書きさせてみよう。
for s in minnichi_sentences:
    print(wakati(s).strip().split(' '))

In [ ]:
# ところで MeCab は入力文を構文解析する場合，解析結果をリストとして返す。
mcb = MeCab.Tagger().parse  # mecab の定義。念のため再定義
s = '何でも薄暗いじめじめした所でニヤーニヤー泣いて居た事丈は記憶して居る。'  # 入力文
print(mcb(s))
print(len(mcb(s).splitlines()[:-1]))


In [ ]:
# 上記を行数を付番して表示してみる。
# `enumerate` を使うと連番を得ることができる
for i, x in enumerate(mcb(s).splitlines()[:-1]):
    print(i, x)


In [ ]:
# 分かち書きされた結果は更に，タブで表層形とその解析結果に分けられるので，分割して見よう。
for i, x in enumerate(mcb(s).splitlines()[:-1]):
    surface, content = x.split('\t')
    print(f'{i:2d}, {surface}: {content})')

In [ ]:
# 必要な情報は，品詞1, 品詞2 および原形だけであるとしよう。
# 上記の出力から，# 品詞1 は 0 番目，品詞2 は 1 番目，原形は 7 番目であることが分かるので
# これを用いることにする
for i, x in enumerate(mcb(s).splitlines()[:-1]):
    surface, content = x.split('\t')
    _x = content.split(',')
    pos1, pos2 = _x[0], _x[1] # それぞれ品詞1, 品詞2
    original_form = surface if len(_x) <= 8 else _x[7]
    print(f'{i:2d} 表層形:{surface}, 品詞1:{pos1} 品詞2:{pos2} 原形:{original_form}')


In [25]:
for i, x in enumerate(mcb(s).splitlines()[:-1]):
    surface, content = x.split('\t')
    _x = content.split(',')
    pos1, pos2 = _x[0], _x[1] # それぞれ品詞1, 品詞2
    original_form = surface if len(_x) <= 8 else _x[7]
    # if len(_x) > 8:
    #     original_form = _x[7]
    # else:
    #     original_form = ""

    print(f'{i:2d} 表層形:{surface}, 品詞1:{pos1} 品詞2:{pos2} 原形:{original_form}')


 0 表層形:何, 品詞1:代名詞 品詞2: 原形:何
 1 表層形:で, 品詞1:助詞 品詞2:格助詞 原形:で
 2 表層形:も, 品詞1:助詞 品詞2:係助詞 原形:も
 3 表層形:薄暗い, 品詞1:形容詞 品詞2:一般 原形:薄暗い
 4 表層形:じめじめ, 品詞1:副詞 品詞2: 原形:じめじめ
 5 表層形:し, 品詞1:動詞 品詞2:非自立可能 原形:為る
 6 表層形:た, 品詞1:助動詞 品詞2: 原形:た
 7 表層形:所, 品詞1:名詞 品詞2:普通名詞 原形:所
 8 表層形:で, 品詞1:助詞 品詞2:格助詞 原形:で
 9 表層形:ニヤーニヤー, 品詞1:名詞 品詞2:普通名詞 原形:ニヤーニヤー
10 表層形:泣い, 品詞1:動詞 品詞2:一般 原形:泣く
11 表層形:て, 品詞1:助詞 品詞2:接続助詞 原形:て
12 表層形:居, 品詞1:動詞 品詞2:非自立可能 原形:居る
13 表層形:た, 品詞1:助動詞 品詞2: 原形:た
14 表層形:事, 品詞1:名詞 品詞2:普通名詞 原形:事
15 表層形:丈, 品詞1:名詞 品詞2:普通名詞 原形:丈
16 表層形:は, 品詞1:助詞 品詞2:係助詞 原形:は
17 表層形:記憶, 品詞1:名詞 品詞2:普通名詞 原形:記憶
18 表層形:し, 品詞1:動詞 品詞2:非自立可能 原形:為る
19 表層形:て, 品詞1:助詞 品詞2:接続助詞 原形:て
20 表層形:居る, 品詞1:動詞 品詞2:非自立可能 原形:居る
21 表層形:。, 品詞1:補助記号 品詞2:句点 原形:。


In [26]:
from termcolor import colored
import numpy as np

for i in range(10):
    wrd = minnichi_vocab[np.random.choice(len(minnichi))]
    print(colored(wrd,'blue', attrs=['bold']), end=": ") 
    #print(colored(minnichi[i]['word'],'blue', attrs=['bold']), end=": ") 
    for _i in minnichi[wrd]['mecab']:
        print('表層形',colored(_i['表層形'], 'green', attrs=['bold']),
              '品詞',  colored(_i['品詞'],  'green', attrs=['bold','blink']), end=" ") 
    print()

父: 表層形 父 品詞  
港: 表層形 港 品詞  
脱ぎます: 表層形 脱ぎ 品詞 五段 表層形 ます 品詞 助動詞 
いす: 表層形 いす 品詞  
課長: 表層形 課長 品詞  
有名: 表層形 有名 品詞  
頼みます: 表層形 頼み 品詞 五段 表層形 ます 品詞 助動詞 
届けます: 表層形 届け 品詞 下一段 表層形 ます 品詞 助動詞 
そろそろ失礼します: 表層形 そろそろ 品詞  表層形 失礼 品詞  表層形 し 品詞 サ行変格 表層形 ます 品詞 助動詞 
残業します: 表層形 残業 品詞  表層形 し 品詞 サ行変格 表層形 ます 品詞 助動詞 


In [27]:
from ccap import ccap_w2v
w2v = ccap_w2v(is2017=True).w2v

In [31]:
#minn_vocab = [minnichi[x] for x in minnichi.keys()]
minn_vocab = minnichi.keys()
minnichi_not_w2v = []
for w in minn_vocab:
    if not w in w2v:
        minnichi_not_w2v.append(w)
    else:
        ; 
        # print(w, end="\t")
print(f'word2vec に存在しない minnichi 単語数:{len(minnichi_not_w2v)}')
print(len(minnichi_not_w2v))
print(f'最初の 3 語を表示: {minnichi_not_w2v[:3]}')
print(f'最後の 3 語を表示: {minnichi_not_w2v[-3:]}')

word2vec に存在しない minnichi 単語数:411
411
最初の 3 語を表示: ['あいます', 'あきらめます', 'あげます']
最後の 3 語を表示: ['飼います', '飾ります', '騒ぎます']


In [32]:
# word2vec に存在しないミンニチ語彙を MeCab によって分解し，各分解した語が word2vec に存在するか否かを調べる
mcb = MeCab.Tagger().parse

for wrd in minnichi_not_w2v:
    surfaces = [ent.split('\t')[0] for ent in mcb(wrd).splitlines()[:-1]]
    for _s in surfaces:
        color = 'red' if not _s in w2v else 'grey'
        if not _s in w2v:
            print(colored((wrd,_s), color, attrs=['bold']), end=" ")    

('おめでとうございます', 'めでとう') ('ごちそうさまでした', 'ちそう') ('初めまして', '初めまして') ('晩ごはん', '晩ごはん') ('片づきます', '片づき') ('雲ります', '雲り') 

In [33]:
wrd = '本'
topn=10
wrd_list = [p[0] for p in w2v.most_similar(wrd,topn=topn)]
print(wrd_list)

['本書', '全', '上記', '同', '当', '本同', '文庫本', '一冊の本', '全巻', '抄出']


In [71]:
def mecab_wakati(phrase:list):
    """mecab を使って分かち書きにする
    colab のインストール状況によっては，分かち書きオプション -Owakati が存在しない。
    根本的な解決は，定義ファイルを書けばよい。
    だが，ここでは標準の MeCab 出力から等価な出力を実現してみた
    
    引数: 
        phrase: list[str]
    
    戻り値: list
    """
    wakati = " ".join(ent.split('\t')[0] for ent in mcb(phrase).splitlines()[:-1])
    return wakati

print(mecab_wakati('これは，私が図書館で借りた本です。'))

def mecab_pos(word:str):
    """mecab を使って単語の品詞情報を得る
    引数: 
        word: str
    戻り値:
        list[str]
    """
    # 次行はトリッキーに見えるかも知れないが，MeCab の出力 `mcb(word)` を
    # 1. 行に区切り (`.splitlines()`)
    # 2. 区切った各項目を更にタブ `('\t')` で区切り
    # 3. その 0 番目の要素である表層形と
    # 4. '(',')` で区切った先頭要素を取り出して
    # 5. タプルにして返す
    # という操作を 1 行でしている
    poses = [(ent.split('\t')[0], ent.split('\t')[1].split(',')[0]) for ent in mcb(word).splitlines()[:-1]]
    return(poses)

print(mecab_pos('これは，私が図書館で借りた本です。'))

これ は ， 私 が 図書 館 で 借り た 本 です 。
[('これ', '代名詞'), ('は', '助詞'), ('，', '補助記号'), ('私', '代名詞'), ('が', '助詞'), ('図書', '名詞'), ('館', '接尾辞'), ('で', '助詞'), ('借り', '動詞'), ('た', '助動詞'), ('本', '名詞'), ('です', '助動詞'), ('。', '補助記号')]


# 2 「みんなの日本語」ファイルの読み込み

In [ ]:
import os
import unicodedata
import glob
import jaconv
import sys
from konoha import SentenceTokenizer
splitter = SentenceTokenizer().tokenize

if isColab:
    from google.colab import files
    uploaded = files.upload()  # `2022_0410minnichi.txt` を指定してアップロードする    
    with open('2022_0410minnichi.txt', 'r') as f:
        minnichi_text = f.readlines()

# コメントアウトしてあるのは，ローカル PC での実行のため，
# # 岩下先生から頂いた「みんなの日本語」データの読み込み
# minnichi_dir = '/Users/asakawa/study/2021jlpt'
# minnichi_files = sorted(glob.glob(os.path.join(minnichi_dir, 'MINNICHI_*.txt')))

# # みんなの日本語テキストを読み込み
# minnichi_text = {}
# for fname in minnichi_files:
#     _fname = os.path.split(fname)[-1].split('.')[0]

#     if not _fname in minnichi_text:
#         minnichi_text[_fname] = []
#     txt = []
#     with open(fname,'r') as f:
#         texts = f.readlines()
        
#         for txt in texts:
#             txt = jaconv.normalize(txt.strip())
#             if len(txt) > 0:
#                 minnichi_text[_fname].append(txt)
                

# _minn_txt = []
# for k, v in minnichi_text.items():
#     for ll in minnichi_text[k]:
#         for _ll in splitter(ll):
#             _minn_txt.append(_ll)

# minnichi_text = _minn_txt            
# print(f'みんなの日本語テキストの総行数:{len(minnichi_text)}')

# 結果を書き出す場合には，次行以下のコメントを削除する
# with open('2022_0410minnichi.txt', 'w') as f:
#     for l in minnichi_text:
#         f.write(l+'\n')

minnichi_pos_dict = {}
for l in minnichi_text:
    for w, pos in mecab_pos(l):
        if pos in minnichi_pos_dict:
            minnichi_pos_dict[pos] += 1
        else:
            minnichi_pos_dict[pos] = 1
        
print(minnichi_pos_dict)

問題作成のための，同義語，反意語を探す工夫


In [3]:
# 次行はローカルな実行環境とクラウド計算環境である google colab との差分を吸収するため
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install --upgrade nltk    
    import nltk
    nltk.download('all')    
    nltk.download('wordnet')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [ ]:
import typing
from nltk.corpus import wordnet as wn

def wordnet_synonym_antonym(word:str,
                            lang:str='jpn'):
    """nltk の wordnet を使って，同義語と反意語の辞書を返す
    引数として word: str をとる。
    オプション lang: ['eng', 'jpn'] 英語か日本語かを指定
    戻り値: dict
        '同義語': list
        '反意語': list
    使用例:
        wordnet_synonym_antonym('英語', lang='jpn')  
        # オプション: lang='jpn' は省略可能
        
        word_synonym_antonym(word='data', lang='eng')
    """
    synonyms, antonyms = [], []

    for syn in wn.synsets(word, lang=lang):
        for l in syn.lemmas(lang=lang):
            synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
                
    return {'同義語':synonyms, '反意語':antonyms}


s = 'これは，私が図書館で借りた本です。'
s_wakati = mecab_wakati(s).split(' ')
for wrd in s_wakati:
    print(f'{wrd}: {wordnet_synonym_antonym(wrd)}')

print('\n---\n')

def wordnet_defs_examples(word:str,
                          lang='jpn'):
    syns = wn.synsets(wrd, lang=lang)
    _def, _exmpl = [], []
    for syn in wn.synsets(word, lang=lang):
        if syn.definition():
            _def.append(syn.definition())
        if syn.examples():
            _exmpl.append(syn.examples())
    return {'定義':_def, '例文':_exmpl}
                          

#wordnet_defs_examples('本')
s = 'これは，私が図書館で借りた本です。'
s_wakati = mecab_wakati(s).split(' ')
for wrd in s_wakati:
    print(f'{wrd}: {wordnet_defs_examples(wrd)}')

In [109]:
syns = wn.synsets('図書', lang='jpn')

for syn in syns:
    print(f'syn:{syn}')
    #print(syn.lemmas(lang='jpn'))
    print(syn.definition())
    print(syn.examples())

syn:Synset('book.n.02')
physical objects consisting of a number of pages bound together
['he used a large book as a doorstop']
syn:Synset('book.n.01')
a written work or composition that has been published (printed on pages bound together)
['I am reading a good book on economics']
